**Tasks 3**

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# Transformasi data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# Memuat dataset MNIST
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform)

# Membuat dataloader
batch_size = 64
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

100%|██████████| 9912422/9912422 [00:00<00:00, 82186227.43it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 62764608.20it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 19437034.92it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13875111.99it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Memulai loop untuk mendapatkan satu batch dari data pelatihan
for images, labels in train_loader:
    pass  # perlu mengganti dengan pemrosesan sesuai kebutuhan

In [ ]:
# Mengimpor modul neural network (nn) dan fungsi aktivasi dari PyTorch
import torch.nn as nn
import torch.nn.functional as F

# Definisi kelas SimpleNN yang merupakan subclass dari nn.Module
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
       # Memanggil konstruktor kelas induk (nn.Module)
        super(SimpleNN, self).__init__()
        # Layer pertama (fully connected) dengan input_size neuron dan hidden_size neuron
        self.fc1 = nn.Linear(input_size, hidden_size)
        # Layer kedua (fully connected) dengan hidden_size neuron dan num_classes neuron
        self.fc2 = nn.Linear(hidden_size, num_classes)

    # Metode forward untuk mendefinisikan aliran data melalui model
    def forward(self, x):
      # Terapkan fungsi aktivasi ReLU pada hasil layer pertama
        x = F.relu(self.fc1(x))
        # Langsung terapkan layer kedua tanpa fungsi aktivasi (untuk klasifikasi)
        x = self.fc2(x)
        # Kembalikan hasilnya
        return x

# Initialize the model
input_size = 28 * 28  # Ukuran gambar MNIST adalah 28x28
hidden_size = 128
num_classes = 10
model = SimpleNN(input_size, hidden_size, num_classes)

In [ ]:
# Definisi fungsi loss menggunakan CrossEntropyLoss
criterion = nn.CrossEntropyLoss()
# Definisi optimizer menggunakan algoritma Adam, dengan learning rate yang telah ditentukan
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Definisi jumlah epoch untuk pelatihan
num_epochs = 5

# Iterasi melalui setiap epoch
for epoch in range(num_epochs):
   # Inisialisasi total loss untuk setiap epoch
    total_loss = 0
    # Iterasi melalui setiap batch dalam data pelatihan menggunakan train_loader
    for images, labels in train_loader:
      # Melakukan flattening pada gambar agar sesuai dengan input model
        images = images.view(-1, 28 * 28)  # Flatten the images
         # Mendapatkan output dari model untuk batch saat ini
        outputs = model(images)
        # Menghitung loss menggunakan fungsi loss (CrossEntropyLoss)
        loss = criterion(outputs, labels)
        # Mengatur gradien parameter model menjadi 0
        optimizer.zero_grad()
        # Melakukan backpropagation untuk menghitung gradien loss
        loss.backward()
        # Melakukan langkah optimasi berdasarkan gradien yang dihitung
        optimizer.step()
        # Menambahkan loss pada batch saat ini ke total loss untuk epoch ini
        total_loss += loss.item()

    # Mencetak rata-rata loss untuk epoch ini
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}")

Epoch 1, Loss: 0.09566045389113538
Epoch 2, Loss: 0.08645134920359596
Epoch 3, Loss: 0.07678621033725859
Epoch 4, Loss: 0.0703288556534801
Epoch 5, Loss: 0.06499459071115636


In [ ]:
# Inisialisasi hitungan untuk evaluasi akurasi model
correct = 0 # jumlah prediksi yang benar
total = 0 # total jumlah gambar yang dievaluasi
model.eval() # Mengubah mode model menjadi evaluasi (non-pelatihan)

# Tidak melakukan perhitungan gradien selama evaluasi
with torch.no_grad():
  # Iterasi melalui setiap batch dalam data uji menggunakan test_loader
    for images, labels in test_loader:
      # Melakukan flattening pada gambar agar sesuai dengan input model
        images = images.view(-1, 28 * 28)
        # Mendapatkan output dari model untuk batch saat ini
        outputs = model(images)
        # Mengambil indeks kelas prediksi dengan nilai tertinggi
        _, predicted = torch.max(outputs.data, 1)
        # Menambahkan jumlah gambar pada batch saat ini ke total
        total += labels.size(0)
        # Menambahkan jumlah prediksi yang benar dengan menghitung kesamaan dengan label sebenarnya
        correct += (predicted == labels).sum().item()

# Menghitung dan mencetak akurasi
print('Akurasi pada gambar uji: {:.2f}%'.format(100 * correct / total))

Akurasi pada gambar uji: 97.11%


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Fungsi untuk mendapatkan prediksi dari model
def get_all_predictions(model, data_loader):
    all_preds = []
    all_labels = []
    model.eval()
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.view(-1, 28 * 28)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted)
            all_labels.extend(labels)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    return all_preds, all_labels

# Mendapatkan prediksi dari model pada data uji
predictions, true_labels = get_all_predictions(model, test_loader)

# Menghitung akurasi
accuracy = accuracy_score(true_labels, predictions)
print('Akurasi: {:.2f}%'.format(accuracy * 100))

# Membuat matriks kebingungan
conf_matrix = confusion_matrix(true_labels, predictions)
print('Confusion Matrix:')
print(conf_matrix)

# Menghitung F1 Score, presisi, dan recall
report = classification_report(true_labels, predictions, target_names=[str(i) for i in range(10)])
print('Laporan Klasifikasi:')
print(report)

Akurasi: 97.11%
Confusion Matrix:
[[ 968    0    0    2    1    3    0    3    1    2]
 [   0 1122    3    2    0    1    3    0    4    0]
 [   4    0 1010    9    1    0    2    2    4    0]
 [   0    0    5  997    0    3    0    1    2    2]
 [   1    0    3    1  950    0    6    2    2   17]
 [   3    0    0   19    1  862    3    0    3    1]
 [   5    2    1    1    2    3  943    0    1    0]
 [   0    6   22   20    0    1    1  967    1   10]
 [   4    2    6   20    5    5    6    2  922    2]
 [   2    3    0   18    6    5    2    2    1  970]]
Laporan Klasifikasi:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98       980
           1       0.99      0.99      0.99      1135
           2       0.96      0.98      0.97      1032
           3       0.92      0.99      0.95      1010
           4       0.98      0.97      0.98       982
           5       0.98      0.97      0.97       892
           6       0.98      0.98  